### Importing necessary modules

In [ ]:
import time
import matplotlib.pyplot as plt
from diffusion import DiffusionEquation

### Utils Methods

In [ ]:
def standard_deviation(arr: list) -> float:
    mean = sum(arr) / len(arr)
    return (sum((x - mean) ** 2 for x in arr) / len(arr)) ** 0.5

In [ ]:
def measure_execution_time(func: callable, n: int) -> list:
    times = []
    for _ in range(n):
        start = time.time()
        func()
        end = time.time()
        times.append(end - start)
    return times

### Measuring execution time for the Sequential implementation

In [ ]:
sequential = DiffusionEquation("../build/libDiffusionEquation.so", N=500)
times = measure_execution_time(sequential.sequential_step, 10000)
print(
    "Final concentration in the center:",
    sequential.concentration_matrix[sequential.N // 2][sequential.N // 2],
)
print("Time elapsed: ", sum(times) / len(times), "+/-", standard_deviation(times))

### Measuring execution time for the OpenMP implementation

In [ ]:
omp = DiffusionEquation("../build/libDiffusionEquation.so", N=500)
times = measure_execution_time(omp.omp_step, 10000)
print(
    "Final concentration in the center:",
    omp.concentration_matrix[omp.N // 2][omp.N // 2],
)
print("Time elapsed: ", sum(times) / len(times), "+/-", standard_deviation(times))

### Firsts Results 

Now that we can measure the execution time of both implementations, let's compare them and check if the OpenMP implementation is faster than the Sequential implementation

We will execute both codes with N increasing from 100 to 1000 with a step of 100 and plot the results. Every test will be executed 10000 times to get a more accurate average time and standard deviation.

In [ ]:
sequential_values = []
for n in range(100, 1001, 100):
    sequential = DiffusionEquation("../build/libDiffusionEquation.so", N=n)
    times = measure_execution_time(sequential.sequential_step, 10000)
    sequential_values.append(sum(times) / len(times))
    
omp_values = []
for n in range(100, 1001, 100):
    omp = DiffusionEquation("../build/libDiffusionEquation.so", N=n)
    times = measure_execution_time(omp.omp_step, 10000)
    omp_values.append(sum(times) / len(times))


In [ ]:
# plot the results together in a single graph
plt.plot(range(100, 1001, 100), sequential_values, label="Sequential")
plt.plot(range(100, 1001, 100), omp_values, label="OMP")
plt.xlabel("Grid size")
plt.ylabel("Time elapsed")
plt.legend()
plt.show()